In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
n = 5
data = pd.DataFrame(columns=['topic', 'text'])
for i in range(1, n+1):
  new_topic = pd.read_csv(f"./drive/MyDrive/requests_gpt_{i}.csv", header=None)
  new_topic.columns = ['topic', 'text']
  data = pd.concat([data, new_topic])
data.drop(data[data['topic'] == 0].index, inplace = True)
print(data['topic'].value_counts())
data

topic
1    155
5    152
4    150
2    140
3    140
Name: count, dtype: int64


,topic,text
0,1,Каким образом можно предъявить иск о задолженн...
1,1,Каковы основания для обращения с иском о задол...
2,1,Какие органы могут принимать и рассматривать и...
3,1,Какие последствия могут возникнуть в случае не...
4,1,Каково количество и размер просроченных платеж...
...,...,...
147,5,Какие последующие шаги следует предпринять пос...
148,5,Как учитывать прецеденты и практику судебных р...
149,5,Поставщик взял деньги и не поставил товар
150,5,Не поставил товар


In [5]:
data['topic'] = data['topic'] - 1

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [17]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации и спецсимволов
    stop_words = set(stopwords.words('russian'))
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return words

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
texts = [preprocess_text(text) for text in data['text']]

In [18]:
vectorizer = TfidfVectorizer(analyzer = preprocess_text)
vectorized_texts = vectorizer.fit_transform(data['text']).toarray()
y = keras.utils.to_categorical(data['topic'], num_classes=5)

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(vectorized_texts, y, test_size=0.15, random_state=54)

In [20]:
X_train.shape

(626, 1164)

In [21]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               596480    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 861701 (3.29 MB)
Trainable params: 861701 (3.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

Epoch 1/10
20/20 [==============================] - 1s 28ms/step - loss: 1.5715 - accuracy: 0.2588 - val_loss: 1.4993 - val_accuracy: 0.5766
Epoch 2/10
20/20 [==============================] - 0s 16ms/step - loss: 1.2877 - accuracy: 0.8195 - val_loss: 1.0458 - val_accuracy: 0.9099
Epoch 3/10
20/20 [==============================] - 0s 16ms/step - loss: 0.5962 - accuracy: 0.9441 - val_loss: 0.4211 - val_accuracy: 0.9099
Epoch 4/10
20/20 [==============================] - 0s 21ms/step - loss: 0.1253 - accuracy: 0.9904 - val_loss: 0.1824 - val_accuracy: 0.9640
Epoch 5/10
20/20 [==============================] - 0s 16ms/step - loss: 0.0335 - accuracy: 1.0000 - val_loss: 0.1319 - val_accuracy: 0.9730
Epoch 6/10
20/20 [==============================] - 0s 16ms/step - loss: 0.0148 - accuracy: 1.0000 - val_loss: 0.1131 - val_accuracy: 0.9730
Epoch 7/10
20/20 [==============================] - 0s 18ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.1020 - val_accuracy: 0.9730
Epoch 8/10
20

In [ ]:
test_data = vectorizer.transform(["Человек не хочет платить мне за аренду квартиры. Что делать?"]).toarray()
np.argmax(model.predict(test_data))

1/1 [==============================] - 0s 25ms/step


1

In [ ]:
test_data = vectorizer.transform(["Меня обокрали. Что делать?"]).toarray()
np.argmax(model.predict(test_data))

1/1 [==============================] - 0s 37ms/step


3

In [ ]:
test_data = vectorizer.transform(["Поставщик обманул на деньги. Как обратиться с этим в суд?"]).toarray()
np.argmax(model.predict(test_data))

1/1 [==============================] - 0s 34ms/step


4